# 使用Redis作为向量数据库

## 非结构化数据问题

如今，组织生成的数据中约有 80% 是非结构化的；没有明确定义的模式或无法重组为熟悉的列格式的数据。非结构化数据的典型示例包括自由格式文本、图像、视频和声音剪辑。预计未来几十年非结构化数据的数量将会增长。

非结构化数据具有高维性和噪声性，使得使用传统方法分析和解释更具挑战性。但它也充满了信息和意义。

传统上，对非结构化数据进行处理以提取特定特征，从而有效地将其转化为结构化数据。一旦进入结构化数据领域，我们就可以使用 SQL 查询（如果存储在关系数据库中）或文本搜索引擎来搜索数据。

将非结构化数据转换为结构化数据的方法存在一些问题。首先，从非结构化数据中提取特征的计算成本可能很高并且容易出错，从而大大延迟了我们有效使用数据的时间。其次，在提取/转换过程中可能会丢失一些保真度和信息，因为独特的潜在特征无法轻松分类或量化。

## 输入向量数据库

处理非结构化数据的一种方法是对数据进行向量化。通过向量化，我们的意思是以某种方式将文本段落、图像、视频或歌曲等内容转换为代表特定数据的平面数字序列。这些向量是 N 维空间中数据的表示。通过向量化，我们能够使用线性代数技术对数据进行比较、分组和操作。这是向量数据库的基础；存储和操作向量的能力。这种方法并不新鲜，并且已经存在很长时间了。今天的不同之处在于生成向量的技术如何进步。

## 使用机器学习embeddings作为向量

将非结构化文本数据转换为向量形式的传统方法包括词袋 (BoW) 和词频-逆文档频率 (TF-IDF)。对于分类数据，one-hot 编码是一种常用的方法。散列和特征提取技术，例如边缘检测、纹理分析或颜色直方图，已用于图像等高维数据。

虽然这些方法本身很强大，但在面对高维和复杂的数据形式（如长文本段落、图像和音频）时却暴露出局限性。例如，考虑如何通过句子重新排列、同义词使用或叙述风格的改变来重组文本段落。这种简单的修改可以有效地避开词袋等技术，防止使用生成的编码的系统识别具有相似含义的文本段落。

这就是机器学习，特别是深度学习的进步发挥作用的地方。机器学习模型促进了嵌入作为一种广泛接受的生成密集、低维向量表示的方法的兴起。给定合适的模型，生成的嵌入可以封装数据中固有的复杂模式和语义，从而克服传统嵌入的局限性。



## 使用bikes数据集来生成embeddings

为了研究向量相似性，我们将使用bikes dataset，这是一个相对简单的合成数据集。该数据集在名为`bikes.json`的JSON文件中包含11条自行车记录，并包含字段`model`、`brand`、`price`、`type`、`specs`和`description`。该`description`字段对包含自行车的自由形式文本描述。

这里使用python来执行redis vss， 安装相关依赖包

```shell
python3 -m venv .venv
source .venv/bin/activate
pip3 install redis
pip3 install pandas
# Generating vector embeddings for text data using SentenceTransformers (pre-trained NLP models)
pip3 install sentence-transformers
```
# reference
1. https://redis.io/docs/interact/search-and-query/search/vectors/
2. https://github.com/RedisVentures/redis-vss-getting-started/blob/main/vector_similarity_with_redis.ipynb